In [7]:
import nltk
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,sent_tokenize
from xlwt import Workbook
import os
import pandas as pd
import boto3

contents = []
bank_name= []
job_url= []
list_unigram=[]
list_bigram=[]
list_trigram=[]
bigram_words= []
bigram_final= []
trigram_final= []
cluster_name= []
cluster_count= []
column_count_value= []
cluster_new= []

#Opens the CSV file from Amazon S3 and appends the values according to the column
client = boto3.client('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
obj_job = client.get_object(Bucket='ads-assignment', Key='Job_Description.csv')
df1 = pd.read_csv(obj_job['Body'])
for i in df1.columns:
    i
for ba in df1['Bank Name']:
     bank_name.append(ba)
for ur in df1['URL']:
     job_url.append(ur)
for de in df1['Description']:
    contents.append(de)

#Opens a new workbook
wb = Workbook()
add = wb.add_sheet('Final_Count', cell_overwrite_ok=True)
row=1
col=6
add.write(0,0, 'Bank Name')
add.write(0,1, 'URL')
add.write(0,2, 'Description')
add.write(0,3, 'Cluster Category')
add.write(0,4, 'Job Type')
add.write(0,5, 'Focused Area')
        
#Writes column names in the excel by taking file from Amazon S3
client = boto3.client('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
obj = client.get_object(Bucket='ads-assignment', Key='Cluster.csv')
df = pd.read_csv(obj['Body'])
df=df.fillna(" ")
for i in df.columns:
    column_count_value.append(i)
    add.write(0,col,i)
    col = col +1
col=6
    
#Writes Bank names in the excel sheet at column 0
for bankname in bank_name[:]:
    bankname
    add.write(row,0,bankname)
    row = row +1    
    
row=1
#Writes URL in the excel sheet at column 1
for job_link in job_url[:]:
    job_link
    add.write(row,1,job_link)
    row=row+1
    
row=1
#Writes Job Desription in the excel sheet at column 2 and does the word count
for jobdata in contents[:]:
    add.write(row,2,jobdata)
    word_tokens1 = word_tokenize(jobdata)
    words1 = [w for w in word_tokens1 if w.isalpha()]
    words_lowercase = [x.lower() for x in words1]
    
#Converting the description to Bigrams and Trigrams
    stop_words = set(stopwords.words('english'))
    nltk_tokens = nltk.word_tokenize(jobdata)
    words2 = [u for u in nltk_tokens if u.isalpha()]
    words2_lowercase = [y.lower() for y in words2]
    for z in words2_lowercase: 
        if z.lower() not in stop_words: 
            bigram_words.append(z)
    bigram = list(nltk.bigrams(bigram_words))
    trigram = list(nltk.trigrams(bigram_words))

#Formatting both the bigrams and trigrams
    for a in bigram:
        b=a[0]+" "+a[1]
        bigram_final.append(b)
    for c in trigram:
        d=c[0]+" "+c[1]+" "+c[2]
        trigram_final.append(d)
        
#Opens the cluster csv files from Amazon S3 and clasifies the job to different clusters based on the max number of words repeated in the cluster        
    client = boto3.client('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
    obj = client.get_object(Bucket='ads-assignment', Key='Cluster.csv')
    df = pd.read_csv(obj['Body'])
    df=df.fillna(" ")
    for i in df.columns:
        i
        for j in df[i]:
            if j != " ":
                splitted = len(j.split())
                if splitted == 1:
                    list_unigram.append(j)
                elif splitted ==2:
                    list_bigram.append(j)
                else:
                    list_trigram.append(j)

        for k in list_unigram:
            count=0
            for num1 in words_lowercase:
                if k == num1:
                    count = count+1
            if count!=0:
                general_count = (i,count)
                general_count1 = (i,k,count)
                cluster_name.append(i)
                cluster_count.append(general_count)
                cluster_new.append(general_count1)
                
        for l in list_bigram:
            count1=0
            for num1_bi in bigram_final:
                if l == num1_bi:
                    count1 = count1+1
            if count1!=0:
                general_count_bi = (i,count1)
                general_count_bi1 = (i,l,count1)
                cluster_name.append(i)
                cluster_count.append(general_count_bi)
                cluster_new.append(general_count_bi1)
                
        for m in list_trigram:
            count2=0
            for num1_tri in trigram_final:
                if m == num1_tri:
                    count2 = count2+1
            if count2!=0:
                general_count_tri = (i,count2)
                general_count_tri1 = (i,m,count2)
                cluster_name.append(i)
                cluster_count.append(general_count_tri)
                cluster_new.append(general_count_tri1)
               
        del list_unigram[:]
        del list_bigram[:]
        del list_trigram[:]
        
#Getting the Cluster names and checking how many words got repeated in that particular cluster    
    cluster_dict = {clus:cluster_name.count(clus) for clus in cluster_name}
    
    data = {k:v for k, v in cluster_dict.items() if v == max(cluster_dict.values())}
    multiple_cluster_dict = {}
    for (key, value) in cluster_count:
        if key in data:
            if key in multiple_cluster_dict:
                multiple_cluster_dict[key].append(value)
            else:
                multiple_cluster_dict[key] = [value]
    
#If only one cluster is having max number of repeated words, it will take that particular cluster for that particular
#job and also it prints the focused area(Word which got repeated max times in the final cluster group)
    if len(data) ==1:
        final_cluster_name=(max(zip(cluster_dict.values(), cluster_dict.keys())))
        word_for_cluster={}
        for clus in cluster_new:
            if final_cluster_name[1] == clus[0]:
                word_for_cluster[clus[1]]=clus[2]
        word_cluster =(max(zip(word_for_cluster.values(), word_for_cluster.keys())))
        add.write(row,3,final_cluster_name[1])
        add.write(row,4,'Fintech')
        add.write(row,5,word_cluster[1])
        
#If two or more cluster are having same number of repeated words, it will take the overall count for those clusters 
#and then prints the cluster which is having max count. Also it prints the focused area(Word which got repeated
#max times in the final cluster group)
    elif len(data) >1:
        sum_value = {k:sum(v) for k,v in multiple_cluster_dict.items()}
        final_cluster_name1=(max(zip(sum_value.values(), sum_value.keys())))
        word_for_cluster1={}
        for clus in cluster_new:
            if final_cluster_name1[1] == clus[0]:
                word_for_cluster1[clus[1]]=clus[2]
        word_cluster1 =(max(zip(word_for_cluster1.values(), word_for_cluster1.keys())))
        add.write(row,3,final_cluster_name1[1])
        add.write(row,4,'Fintech')
        add.write(row,5,word_cluster1[1])
        
    else:
        add.write(row,3,'Others')
        add.write(row,4,'Non-Fintech')
        add.write(row,5,'Others')


#Creating a dictionary with all the cluster words and assigning the value 0
    cluster_dict_column = {clus_name:0 for clus_name in column_count_value}

    def replace_value(key_to_find, new_value):
        '''This function helps us to replace the total number of counts for each Cluster depending on the output(cluster_dict). 
        It will update the dictionary- cluster_dict_column. Since it is used to write data in final csv file'''
        for key in cluster_dict_column.keys():
            if key == key_to_find:
                cluster_dict_column[key] = new_value

    for (key, value) in cluster_dict.items():
        replace_value(key, value)
        
#Printing the count value for each cluster          
    for val,num in cluster_dict_column.items():
        add.write(row,col,num)
        col=col+1
        if col == 18:
            col=6
            row=row+1
        
    del words_lowercase[:]
    del cluster_name[:]
    del cluster_new[:]
    del cluster_count[:]
    del bigram_final[:]
    del bigram_words[:]
    del bigram[:]
    del trigram[:]
    del trigram_final[:]
    
wb.save(os.getcwd()+'/Part2.csv')

s3 = boto3.resource('s3',aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)
bucket = s3.Bucket('ads-assignment')
s3.Object('ads-assignment', 'Part2.csv').put(Body=open(os.getcwd()+'/Part2.csv', 'rb'))

